In [28]:
import numpy as np  
import pandas as pd  
from pandas_datareader import data as web  
from scipy.stats import norm 
import matplotlib.pyplot as plt  
%matplotlib inline

In [29]:
ticker = 'PG'  
data = pd.DataFrame()
data[ticker] = web.DataReader(ticker, data_source='yahoo', start='2007-1-1', end='2017-3-21')['Close']

log_returns = np.log(1 + data.pct_change())

<br /><br />
$$
{\LARGE S_t = S_{t-1} \mathbin{\cdot} e^{((r - \frac{1}{2} \cdot stdev^2) \mathbin{\cdot} \delta_t + stdev \mathbin{\cdot} \sqrt{\delta_t} \mathbin{\cdot} Z_t)}  }
$$
<br /><br />

In [30]:
r = 0.025
stdev = log_returns.std() * 252 ** 0.5

In [31]:
type(stdev)

pandas.core.series.Series

In [32]:
stdev = stdev.values
stdev

array([0.17833648])

In [33]:
T = 1.0
t_intervals = 250
delta_t = T / t_intervals
iterations = 10000

In [34]:
Z = np.random.standard_normal((t_intervals + 1, iterations))  
S = np.zeros_like(Z) 
S0 = data.iloc[-1]  
S[0] = S0

<br /><br />
$$
{\LARGE S_t = S_{t-1} \mathbin{\cdot} e^{((r - \frac{1}{2} \cdot stdev^2) \mathbin{\cdot} \delta_t + stdev \mathbin{\cdot} \sqrt{\delta_t} \mathbin{\cdot} Z_t)}  }
$$
<br /><br />

In [35]:
for t in range(1, t_intervals + 1):
    S[t] = S[t-1] * np.exp((r - 0.5 * stdev ** 2) * delta_t + stdev * delta_t ** 0.5 * Z[t])
    
S

array([[ 90.98999786,  90.98999786,  90.98999786, ...,  90.98999786,
         90.98999786,  90.98999786],
       [ 90.34187523,  90.60124854,  90.10453026, ...,  91.38732261,
         91.59155507,  90.93287212],
       [ 90.91264502,  89.82871036,  89.97700317, ...,  89.63681708,
         90.65798595,  90.16476425],
       ...,
       [126.22356574,  64.86647936,  91.80262991, ..., 119.89403338,
         56.78063986, 105.02410287],
       [126.27736452,  65.66794004,  91.11260211, ..., 121.5411718 ,
         58.19767407, 104.21695396],
       [125.86766232,  64.77539212,  89.89604489, ..., 121.89843694,
         57.75229552, 105.08210142]])

In [36]:
S.shape

(251, 10000)

In [37]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [38]:
df = pd.DataFrame(data = S)
new_df = df.loc[:,:10]
new_df.iplot()

     **Call Option**
    Buy if:        S-K > 0
    Don't buy if:  S-K < 0

In [39]:
#S in this case is the new df as well
p = np.maximum(S[-1] - 110, 0)

In [40]:
p

array([15.86766232,  0.        ,  0.        , ..., 11.89843694,
        0.        ,  0.        ])

In [41]:
p.shape

(10000,)

$$
C = \frac{exp(-r \cdot T) \cdot \sum{p_i}}{iterations}
$$

In [42]:
#This number should be relativily close to the number in Merton worksheet
C = np.exp(-r * T) * np.sum(p) / iterations
C

1.6798192050031129